In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.keys import Keys


from selenium import webdriver
from selenium.webdriver.common.keys import Keys


## Scrapper de Carrefour

In [3]:


def scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight,bouton_cookies = True):
    # Initialiser le navigateur Chrome
    #chrome_path = "/Users/victor/dev/chromedriver/ChromeDriver 114.0.5735.90/chromedriver" 
    #chrome_service = ChromeService(chrome_path)
    #driver = webdriver.Chrome()


    options = webdriver.SafariOptions()
    driver = webdriver.Safari(options=options)

    # Accéder à l'URL
    driver.get(url)

    time.sleep(5)

    # scraped_product_names stock les produits déja récup par le scrap
    scraped_product_names = set()
    if bouton_cookies:
    # scipt poue le bouton des cookies
        driver.execute_script("""
            var rejectButton = document.getElementById("onetrust-reject-all-handler");
            if (rejectButton) {
                rejectButton.click();
            } else {
                console.log("Le bouton n'a pas été trouvé.");
            }
        """)

    # liste pour stocker les données
    data = []



    i = 0
    # Boucle pour scraper le nombre spécifié de produits
    while len(data) < num_products_to_scrape:
        # Récupérer les éléments de la classe "product-grid-item"
        product_cards = driver.find_elements(By.CLASS_NAME, name_class_product_grid_item)

        # Parcourir les cartes de produits
        for product_card in product_cards:
            try:
                # Récupérer le nom du produit
                name = product_card.find_element(By.CLASS_NAME, name_class_product_card_title_text).text if product_card.find_elements(By.CLASS_NAME, name_class_product_card_title_text) else None

                # Vérifier si le produit a déjà été récupéré
                if name not in scraped_product_names:
                    #recuperer le lien du produit
                    product_link = product_card.find_element(By.CLASS_NAME, name_product_link).get_attribute("href") if product_card.find_elements(By.CLASS_NAME, name_product_link) else None

                    # Récupérer le lien de l'image
                    #image_link = product_card.find_element(By.CLASS_NAME, "image image--hidden product-card-image__image ls-is-cached image--loaded").get_attribute("src") if product_card.find_elements(By.CLASS_NAME, "main-horizontal__image") else None
                    image_element = driver.find_element(By.CLASS_NAME, name_image)
                    image_link = image_element.get_attribute('src')
                    # Récupérer l'origine du produit
                    origin = product_card.find_element(By.CLASS_NAME, name_origin).text if product_card.find_elements(By.CLASS_NAME, name_origin) else None

                    # Récupérer le prix
                    price = product_card.find_element(By.CLASS_NAME, name_price).text if product_card.find_elements(By.CLASS_NAME, name_price) else None

                    # Récupérer le prix à l'unité
                    price_per_unit = product_card.find_element(By.CLASS_NAME, name_price_per_unit).text if product_card.find_elements(By.CLASS_NAME, name_price_per_unit) else None

                    #recuperer le poids
                    weight = product_card.find_element(By.CLASS_NAME, name_weight).text if product_card.find_elements(By.CLASS_NAME, name_weight) else None


                    # Ajouter les données à la liste
                    data.append({
                        "lien du produit": product_link,
                        'Nom': name,
                        'Image': image_link,
                        'Origine': origin,
                        'Prix': price,
                        'Prix à l\'unité': price_per_unit,
                        'Poids': weight,
                    })

                    # Ajouter le nom du produit à l'ensemble des produits déjà récupérés
                    scraped_product_names.add(str(name)+'_'+str(i))
                    i += 1
            except Exception as e:
                print(f"Erreur lors du traitement d'une carte de produit : {e}")
        # Ajouter le scroll en bas de la page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight - 1500);")

        time.sleep(1)
        try:
            # Vérifier si le bouton "Voir plus" est présent
            see_more_button = driver.find_element(By.CLASS_NAME, "pagination__button-wrap").find_element(By.TAG_NAME, "button")
            print("Il y a un bouton")
            # Cliquer sur le bouton "Voir plus"
            see_more_button.click()
            # Attendre un court instant pour le chargement
            time.sleep(2)
        except:
            print("Il n'y a plus de bouton")
            # Quitter la boucle s'il n'y a plus de bouton "Voir plus"
            break

    # Fermer le navigateur
    driver.quit()

    # on créé un DataFrame à partir des données
    df = pd.DataFrame(data)

    return df




In [8]:
#POUR CARREFOUR

url = "https://www.carrefour.fr/r/fruits-et-legumes/legumes"
num_products_to_scrape = 1500

# grille de produits
name_class_product_grid_item = "product-grid-item"
#nom du produit
name_class_product_card_title_text = "product-card-title__text"
#lien du produit
name_product_link = "product-card-image"
#image du produit
name_image = "product-card-image__image"
#origine du produit
name_origin = "ds-product-card-refonte__origin-text"    
#prix du produit
name_price = "product-price__amount-value"
#prix à l'unité du produit
name_price_per_unit = "ds-product-card-refonte__perunitlabel"
#poids du produit
name_weight = "product-card-title__weight"

df = scrap_function(url, num_products_to_scrape, name_class_product_grid_item, name_class_product_card_title_text, name_product_link, name_image, name_origin, name_price, name_price_per_unit, name_weight)

Il y a un bouton
Il y a un bouton


KeyboardInterrupt: 

df

In [9]:
df

,lien du produit,Nom,Image,Origine,Prix,Prix à l'unité,Poids
0,https://www.carrefour.fr/p/avocat-bio-carrefou...,Avocat Bio CARREFOUR BIO,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,69",1.69 € / U,None
1,https://www.carrefour.fr/p/avocat-hass-bio-352...,Avocat Hass Bio,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,69",1.69 € / U,None
2,https://www.carrefour.fr/p/avocat-hass-bio-352...,Avocat Hass Bio,https://media.carrefour.fr/medias/6605ad939302...,KENYA,"1,59",1.59 € / U,None
3,https://www.carrefour.fr/p/asperges-moyennes-v...,Asperges moyennes violettes vrac,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"15,99",15.99 € / KG,None
4,https://www.carrefour.fr/p/asperges-violettes-...,Asperges violettes,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"5,90",11.80 € / KG,None
...,...,...,...,...,...,...,...
507,https://www.carrefour.fr/p/brocoli-carrefour-3...,Brocoli CARREFOUR,https://media.carrefour.fr/medias/6605ad939302...,origine UE et Hors U.E.,"1,49",1.49 € / U,None
508,https://www.carrefour.fr/p/brocoli-bio-3270190...,Brocoli bio,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"3,29",3.29 € / U,None
509,https://www.carrefour.fr/p/brocoli-bio-3276550...,Brocoli Bio,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,99",1.99 € / U,None
510,None,None,https://media.carrefour.fr/medias/6605ad939302...,None,None,None,None


In [7]:
#prendre dans df les produits qui ont un nom qui contient "Brocoli" ou "brocoli" et qui n'est pas nan
df_brocoli = df[df['Nom'].str.contains("Brocoli|brocoli", na=False)]
df_brocoli

,lien du produit,Nom,Image,Origine,Prix,Prix à l'unité,Poids
23,https://www.carrefour.fr/p/brocoli-32765501958...,Brocoli,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,99",1.99 € / U,None
24,https://www.carrefour.fr/p/brocoli-32765502019...,Brocoli,https://media.carrefour.fr/medias/6605ad939302...,ORIGINE C.E.E.,"1,59",3.53 € / KG,None
25,https://www.carrefour.fr/p/brocoli-bio-carrefo...,Brocoli Bio CARREFOUR BIO,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"2,99",2.99 € / U,None
26,https://www.carrefour.fr/p/brocoli-32765598255...,Brocoli,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,29",1.29 € / U,None
27,https://www.carrefour.fr/p/brocoli-carrefour-3...,Brocoli CARREFOUR,https://media.carrefour.fr/medias/6605ad939302...,origine UE et Hors U.E.,"1,49",1.49 € / U,None
...,...,...,...,...,...,...,...
505,https://www.carrefour.fr/p/brocoli-bio-carrefo...,Brocoli Bio CARREFOUR BIO,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"2,99",2.99 € / U,None
506,https://www.carrefour.fr/p/brocoli-32765598255...,Brocoli,https://media.carrefour.fr/medias/6605ad939302...,ESPAGNE,"1,29",1.29 € / U,None
507,https://www.carrefour.fr/p/brocoli-carrefour-3...,Brocoli CARREFOUR,https://media.carrefour.fr/medias/6605ad939302...,origine UE et Hors U.E.,"1,49",1.49 € / U,None
508,https://www.carrefour.fr/p/brocoli-bio-3270190...,Brocoli bio,https://media.carrefour.fr/medias/6605ad939302...,FRANCE,"3,29",3.29 € / U,None


## Ce code Fonctionne bien pour scraper les infos des légumes carrefour

In [10]:

df.to_csv('/Users/victor/Library/CloudStorage/OneDrive-DeVinci/esilv/s9/projet_webscrapping/légumes_data_carrefour.csv', index=False)